In [ ]:
# mecab과 konlpy는 설치 되어있어야 합니다.
# 이후에 numpy도 필요하긴 해요.
# 행렬곱이나 전치, list로 할라니 성능도 안나고 문제네요...
from konlpy.tag import Mecab, Okt

with open("sample.txt", "rt", encoding='utf-8') as f:
    text_list = f.readlines()

def clean_text(text):
    cleaned_text = re.sub('[a-zA-z0-9]','',text)
    cleaned_text = re.sub('[\{\}\[\]\/?,;:|\‘’)*~`!^\-_+<>@\#$%&\\\=\(\'\"\♥\♡\ㅋ\ㅠ\ㅜ\ㄱ\ㅎ\ㄲ\ㅡ]','',cleaned_text)
    return cleaned_text

# 일단 영어나 숫자, 특수문자는 빼고 생각해봅시다 우리.
text_list = list(map(lambda x :Okt().normalize(clean_text(x)).replace('  ',' ').replace('\n',''), text_list))

In [ ]:
# 형태소는 제가 제일 좋아하는 mecab으로!
mecab = Mecab()
morphs_list = list(map(lambda x:mecab.morphs(x), text_list))

In [ ]:
# vocab dict을 만들건데, 일단은 count도 중요할까 싶어서 이렇게 짜봤습니다.
import collections
morphs_dict = collections.Counter()
for doc in morphs_list:
    morphs_dict.update(doc)
morphs_dict = dict(sorted(morphs_dict.items()))
vocab_list = list(morphs_dict.keys())

In [ ]:
# 마르코프 = 현재 상태만이 미래에 영향을 미친다!
markov_dict = collections.defaultdict(list)
for i in range(len(morphs_list)):
    for j in range(len(morphs_list[i])-1):
        markov_dict[morphs_list[i][j]].append(morphs_list[i][j+1]) 

In [ ]:
# 확률을 구하기 위한 일단 숫자셈
for key in markov_dict:
    markov_counter = collections.Counter()
    markov_counter.update(markov_dict[key])
    markov_dict[key]=dict(markov_counter)

In [ ]:
# transition 배열을 나중에 만들기 위해, 없는 vocab에 대해서 0으로 채우고
# transition이 pandas 형태가 아니기 때문에, vocab_list와 1:1 index mapping을 위해서 동일하게 정렬을 해줍니다.
# transition은 dict X dict 정방 형태이므로, key도 value도 전부 order 해줘야 예외가 안생깁니다.
for vocab in vocab_list:
    for markov in markov_dict.keys():
        try:
            markov_dict[markov][vocab] = markov_dict[markov][vocab]
        except KeyError:
            markov_dict[markov][vocab]=0
markov_dict = dict(sorted(markov_dict.items()))
for markov in markov_dict.keys():
    markov_dict[markov] = dict(sorted(markov_dict[markov].items()))

In [ ]:
# 현재 상태와 곱해서 뭔가 다음에 나올 문자열에 대한 확률을 보고 싶을 수도 있으니까 한번 transition도 만들고
# 현재 상태도 곱해봅시다!
transition = list()
for i in markov_dict.keys():
    transition.append(list(map(lambda x:x/sum(markov_dict[i].values()), markov_dict[i].values())))

In [ ]:
# 문장은 '.'로 끝난다고 생각하고, 그냥 랜덤으로 선택하면서 문장을 만들어보자
# 뭐 다만 그나마 랜덤으로 선택할때, 좀 많이 나올만한 애들로 선택해볼까?
def make_sentence(input_text):
    import random
    next_text = input_text
    whole_word = input_text
    while next_text!='.':
        random_list = list()
        random_weight = list()
        for i, val in markov_dict[next_text].items():
            if val>0:
                random_list.append(i)
                random_weight.append(val)
        next_text = random.choices(random_list, weights=random_weight)[0]
        whole_word = whole_word + ' ' + next_text
    return whole_word

In [ ]:
# 결과는?
for _ in range(20):
    print(make_sentence('마르코프'))

In [ ]:
# 이쯤되면 전이확률 행렬은 어떤 값이 나올지, 누구나 알 수 있을듯.
import numpy as np
x_list = np.array([0 for _ in range(len(transition[0]))])
x_list[vocab_list.index('마르코프')]=1
np_transition = np.array(transition)

In [ ]:
# 중요한 사실은 기준을 row로 잡았느냐, col 로 잡느냐 이다.
# 어케 잡느냐에 따라서, 과거/미래 값이 나오게 되니까...!
x_list_t = x_list.reshape(-1,1)
temp = np_transition.transpose()

In [ ]:
next_p = np.matmul(temp,x_list_t)

In [ ]:
# 뭐가 나왔을까...!?
for i,val in enumerate(next_p.flatten()):
    if val>0:
        print(list(morphs_dict.keys())[i],val)